In [1]:
# Importing the necessary packages
import pandas as pd
import numpy as np
import keras
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [2]:
# load pima indians dataset
dataset = np.loadtxt('https://raw.githubusercontent.com/mchandak/DS_Repo10/main/Data/pima-indians-diabetes.data.csv', delimiter=",")

In [3]:
X = dataset[:,0:8]
y = dataset[:,8]

In [4]:
dataset

array([[  6.   , 148.   ,  72.   , ...,   0.627,  50.   ,   1.   ],
       [  1.   ,  85.   ,  66.   , ...,   0.351,  31.   ,   0.   ],
       [  8.   , 183.   ,  64.   , ...,   0.672,  32.   ,   1.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,   0.245,  30.   ,   0.   ],
       [  1.   , 126.   ,  60.   , ...,   0.349,  47.   ,   1.   ],
       [  1.   ,  93.   ,  70.   , ...,   0.315,  23.   ,   0.   ]])

In [5]:
# Standardization
a = StandardScaler()
a.fit(X)
X_standardized = a.transform(X)

In [6]:
pd.DataFrame(X_standardized).describe()

,0,1,2,3,4,5,6,7
count,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02,7.680000e+02
mean,-8.789266e-17,-9.251859e-18,1.503427e-17,8.211024e-17,-4.972874e-17,2.960595e-15,1.905883e-15,2.081668e-16
std,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00,1.000652e+00
min,-1.141852e+00,-3.783654e+00,-3.572597e+00,-1.288212e+00,-6.928906e-01,-4.060474e+00,-1.189553e+00,-1.041549e+00
25%,-8.448851e-01,-6.852363e-01,-3.673367e-01,-1.288212e+00,-6.928906e-01,-5.955785e-01,-6.889685e-01,-7.862862e-01
50%,-2.509521e-01,-1.218877e-01,1.496408e-01,1.545332e-01,-4.280622e-01,9.419788e-04,-3.001282e-01,-3.608474e-01
75%,6.399473e-01,6.057709e-01,5.632228e-01,7.190857e-01,4.120079e-01,5.847705e-01,4.662269e-01,6.602056e-01
max,3.906578e+00,2.444478e+00,2.734528e+00,4.921866e+00,6.652839e+00,4.455807e+00,5.883565e+00,4.063716e+00


#### ****Tuning of Hyperparameters :- ****

1.   Batch Size
2.   Epochs



In [7]:
# Importing the necessary packages
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
#from keras.optimizers import Adam
#from keras.optimizers import adam_v2
from tensorflow.keras.optimizers import Adam

In [8]:
# create model
def create_model():
    model = Sequential(name='Hypterparameter-Tuning-Dummy')
    model.add(Dense(12, input_dim=8, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(8,kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
    
    adam=Adam(learning_rate=0.01)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [9]:
# Create the model
model = KerasClassifier(build_fn = create_model,verbose = 0)
# Define the grid search parameters
batch_size = [10,20,40]
epochs = [50,100,150]
# Make a dictionary of the grid search parameters
param_grid = dict(batch_size = batch_size,epochs = epochs)
# Build and fit the GridSearchCV
grid = GridSearchCV(estimator = model,param_grid = param_grid,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START batch_size=10, epochs=50....................................
[CV 1/5; 1/9] END .....batch_size=10, epochs=50;, score=0.740 total time=   5.4s
[CV 2/5; 1/9] START batch_size=10, epochs=50....................................
[CV 2/5; 1/9] END .....batch_size=10, epochs=50;, score=0.701 total time=   5.3s
[CV 3/5; 1/9] START batch_size=10, epochs=50....................................
[CV 3/5; 1/9] END .....batch_size=10, epochs=50;, score=0.773 total time=   5.2s
[CV 4/5; 1/9] START batch_size=10, epochs=50....................................
[CV 4/5; 1/9] END .....batch_size=10, epochs=50;, score=0.824 total time=   4.9s
[CV 5/5; 1/9] START batch_size=10, epochs=50....................................
[CV 5/5; 1/9] END .....batch_size=10, epochs=50;, score=0.765 total time=   5.2s
[CV 1/5; 2/9] START batch_size=10, epochs=100...................................
[CV 1/5; 2/9] END ....batch_size=10, epochs=100;,

In [10]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7670146822929382, using {'batch_size': 40, 'epochs': 50}
0.7605042099952698,0.040135676318675935 with: {'batch_size': 10, 'epochs': 50}
0.7566420555114746,0.051402634628635695 with: {'batch_size': 10, 'epochs': 100}
0.7656990051269531,0.03159229804539973 with: {'batch_size': 10, 'epochs': 150}
0.7514217853546142,0.048564283549136895 with: {'batch_size': 20, 'epochs': 50}
0.751328420639038,0.026201914995678224 with: {'batch_size': 20, 'epochs': 100}
0.7578134298324585,0.040586481386219966 with: {'batch_size': 20, 'epochs': 150}
0.7670146822929382,0.03731961109561036 with: {'batch_size': 40, 'epochs': 50}
0.7527035117149353,0.05072736132748933 with: {'batch_size': 40, 'epochs': 100}
0.7422205448150635,0.0262977690070821 with: {'batch_size': 40, 'epochs': 150}


#### Tuning of Hyperparameters:- Learning rate and Drop out rate

In [11]:
from keras.layers import Dropout

# Defining the model

def create_model(learning_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(8,input_dim = 8,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(12,input_dim = 8,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(learning_rate = learning_rate)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 50)

# Define the grid search parameters

learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]

# Make a dictionary of the grid search parameters

param_grids = dict(learning_rate = learning_rate,dropout_rate = dropout_rate)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 1/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.753 total time=   2.2s
[CV 2/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 2/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.721 total time=   2.2s
[CV 3/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 3/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.773 total time=   2.2s
[CV 4/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 4/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.843 total time=   2.4s
[CV 5/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 5/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.771 total time=   2.2s
[CV 1/5; 2/9] START dropout_rate=0.0, learning_rate=0.01........................
[CV 1/5; 

In [12]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7748238682746887, using {'dropout_rate': 0.1, 'learning_rate': 0.001}
0.7722264647483825,0.04009141855413296 with: {'dropout_rate': 0.0, 'learning_rate': 0.001}
0.7578728437423706,0.029561298325359524 with: {'dropout_rate': 0.0, 'learning_rate': 0.01}
0.7344707608222961,0.04843749517686428 with: {'dropout_rate': 0.0, 'learning_rate': 0.1}
0.7748238682746887,0.03964970078414785 with: {'dropout_rate': 0.1, 'learning_rate': 0.001}
0.7526780486106872,0.03905937483735646 with: {'dropout_rate': 0.1, 'learning_rate': 0.01}
0.7474832534790039,0.037022898421406186 with: {'dropout_rate': 0.1, 'learning_rate': 0.1}
0.7605296850204468,0.04229944983046996 with: {'dropout_rate': 0.2, 'learning_rate': 0.001}
0.7540106892585754,0.053425112720747896 with: {'dropout_rate': 0.2, 'learning_rate': 0.01}
0.748790442943573,0.03616366082867292 with: {'dropout_rate': 0.2, 'learning_rate': 0.1}


#### Tuning of Hyperparameters:- Activation Function and Kernel Initializer

In [13]:
# Defining the model

def create_model(activation_function,init):
    model = Sequential()
    model.add(Dense(8,input_dim = 8,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.2))
    model.add(Dense(12,input_dim = 8,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.2))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(learning_rate = 0.001)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 50)

# Define the grid search parameters
activation_function = ['softmax','relu','tanh']
init = ['uniform','normal','zero']

# Make a dictionary of the grid search parameters
param_grids = dict(activation_function = activation_function,init = init)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)



Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START activation_function=softmax, init=uniform...................
[CV 1/5; 1/9] END activation_function=softmax, init=uniform;, score=0.649 total time=   2.5s
[CV 2/5; 1/9] START activation_function=softmax, init=uniform...................
[CV 2/5; 1/9] END activation_function=softmax, init=uniform;, score=0.584 total time=   2.7s
[CV 3/5; 1/9] START activation_function=softmax, init=uniform...................
[CV 3/5; 1/9] END activation_function=softmax, init=uniform;, score=0.630 total time=   3.3s
[CV 4/5; 1/9] START activation_function=softmax, init=uniform...................
[CV 4/5; 1/9] END activation_function=softmax, init=uniform;, score=0.745 total time=   2.8s
[CV 5/5; 1/9] START activation_function=softmax, init=uniform...................
[CV 5/5; 1/9] END activation_function=softmax, init=uniform;, score=0.647 total time=   2.5s
[CV 1/5; 2/9] START activation_function=softmax, init=normal..........

In [14]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7669892191886902, using {'activation_function': 'tanh', 'init': 'uniform'}
0.6511586427688598,0.05244526932680711 with: {'activation_function': 'softmax', 'init': 'uniform'}
0.6616161584854126,0.0717822440570832 with: {'activation_function': 'softmax', 'init': 'normal'}
0.6511586427688598,0.05244526932680711 with: {'activation_function': 'softmax', 'init': 'zero'}
0.7592224836349487,0.04551555406902966 with: {'activation_function': 'relu', 'init': 'uniform'}
0.7540107011795044,0.044084743622199205 with: {'activation_function': 'relu', 'init': 'normal'}
0.6511586427688598,0.05244526932680711 with: {'activation_function': 'relu', 'init': 'zero'}
0.7669892191886902,0.036149489629613486 with: {'activation_function': 'tanh', 'init': 'uniform'}
0.7669892191886902,0.03729767766423352 with: {'activation_function': 'tanh', 'init': 'normal'}
0.6511586427688598,0.05244526932680711 with: {'activation_function': 'tanh', 'init': 'zero'}


#### Tuning of Hyperparameter :-Number of Neurons in activation layer

In [15]:
# Defining the model

def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 8,kernel_initializer = 'uniform',activation = 'linear'))
    model.add(Dropout(0.2))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = 'uniform',activation = 'linear'))
    model.add(Dropout(0.2))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(learning_rate = 0.001)
    #adam = Adam(lr = 0.001)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 50)

# Define the grid search parameters

neuron1 = [4,8,16]
neuron2 = [2,4,8]

# Make a dictionary of the grid search parameters

param_grids = dict(neuron1 = neuron1,neuron2 = neuron2)

# Build and fit the GridSearchCV

grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X_standardized,y)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 1/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.786 total time=   2.4s
[CV 2/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 2/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.721 total time=   2.4s
[CV 3/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 3/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.766 total time=   2.7s
[CV 4/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 4/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.830 total time=   2.4s
[CV 5/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 5/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.758 total time=   2.4s
[CV 1/5; 2/9] START neuron1=4, neuron2=4........................................
[CV 1/5; 2/9] END .........neuron1=4, neuron2=4;,

In [16]:
# Summarize the results
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.7735166788101197, using {'neuron1': 8, 'neuron2': 4}
0.7721925139427185,0.03579823818066126 with: {'neuron1': 4, 'neuron2': 2}
0.7696035981178284,0.03512968658388637 with: {'neuron1': 4, 'neuron2': 4}
0.7708938121795654,0.03637661717740639 with: {'neuron1': 4, 'neuron2': 8}
0.7682794332504272,0.03982754094833611 with: {'neuron1': 8, 'neuron2': 2}
0.7735166788101197,0.03478739227860509 with: {'neuron1': 8, 'neuron2': 4}
0.7696120858192443,0.036947040093127693 with: {'neuron1': 8, 'neuron2': 8}
0.7682794332504272,0.03358195514106587 with: {'neuron1': 16, 'neuron2': 2}
0.7722179770469666,0.03343295092726336 with: {'neuron1': 16, 'neuron2': 4}
0.7669977068901062,0.03697344734210317 with: {'neuron1': 16, 'neuron2': 8}


#### Training model with optimum values of Hyperparameters

In [17]:
from sklearn.metrics import classification_report, accuracy_score

# Defining the model

def create_model():
    model = Sequential()
    model.add(Dense(16,input_dim = 8,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(4,input_dim = 16,kernel_initializer = 'uniform',activation = 'tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(learning_rate = 0.001)
    model.compile(loss = 'binary_crossentropy',optimizer = adam,metrics = ['accuracy'])
    return model

# Create the model

model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 10)

# Fitting the model

model.fit(X_standardized,y)

# Predicting using trained model

y_predict = model.predict(X_standardized)

# Printing the metrics
print(accuracy_score(y,y_predict))

24/24 [==============================] - 0s 2ms/step
0.7786458333333334
